In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import random
import operator
import copy
import math
import time

In [110]:
class NeuralNetwork(object):
    def __init__(self, sizes):
        self.n_layers = len(sizes)
        #self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = self.get_network(sizes)
        #print('w',self.weights)
    def get_network(self, sizes):
        indiv = []
        for i in range(self.n_layers-1):
            w = np.random.rand(sizes[i+1],sizes[i])
            indiv.append(copy.deepcopy(w))
        return indiv
    def feedforward(self, a):
        for i, w in enumerate(self.weights,1):
            a = self.activation_function(np.matmul(w,a),i)
        return np.argmax(a)
    def activation_function(self, r, i):
        if i==self.n_layers-1:
            return self.softmax(r)
        else:
            return self.relu(r)
    def softmax(self,indiv):
        softmax =np.exp(indiv)/sum(np.exp(indiv))
        return softmax
    def relu(self,inputs):
        res = [max(0,i) for i in inputs]
        return res
    def accuracy(self, preds,y):
        correct = 0
        for x in range(len(preds)):
            if preds[x] == np.argmax(y[x]):
                correct+=1
        return correct/len(preds)
    def predict(self, X):
        res =[]
        for i in (X):
            res.append(copy.deepcopy(self.feedforward(i)))
        return res

In [148]:
class GA:
    def __init__(self, N, net_size, Pc, Pm, X, y, n_casillas):
        self.N = N
        self.net_size = net_size
        self.nets = [NeuralNetwork(self.net_size) for i in range(self.N)]
        self.Pm = Pm
        self.Pc = Pc
        self.X = X[:]
        self.y = y[:]
        self.n_casillas = n_casillas
    def fitness(self,W, X, y):
        preds = W.predict(X)
        acc = W.accuracy(preds,y)
        return acc
    def get_aptitudes(self):
        return [self.fitness(w, self.X, self.y) for w in self.nets]
            
        
    def elitismo(self,casillas,aptitudes):
        elit = self.n_casillas -np.sum(casillas)
        bestIndividuo =  np.argmax(aptitudes)
        casillas[bestIndividuo] += elit
        return casillas

    def getCasillasPorIndividuo(self, aptitudes):
        apt2 = [i**2 for i in aptitudes]
        total = np.sum(apt2)
        casillasIndiv = []
        j = 0
        if total==0:
            casillasIndiv =[math.floor(self.n_casillas / len(apt2)) for i in apt2]
        else:
            for i in apt2:
                casillasIndiv.append(math.floor((i / total) * 100))
        if np.sum(casillasIndiv) < self.n_casillas:
            casillasIndiv = self.elitismo(casillasIndiv,aptitudes)
        return  casillasIndiv

    def crearRuleta(self,aptitudes):
        casillasIndiv = self.getCasillasPorIndividuo(aptitudes)
        ruleta = np.zeros(len(aptitudes))
        alfa =0
        for i in range(0, len(ruleta)):
            if casillasIndiv[i] == 0:
                ruleta[i] =  alfa + casillasIndiv[i]
            else:
                ruleta[i] = alfa + casillasIndiv[i] -1
            alfa += casillasIndiv[i]
        return ruleta

    def seleccion(self):
        apt = self.get_aptitudes()
        ruleta = self.crearRuleta(apt)
        gSelec = []
        for i in range (0,self.N):
            tirada = random.randint(0,self.n_casillas-1)
            for j in range(0, len(ruleta)):
                if tirada <= ruleta[j]:
                    gSelec.append(self.nets[j])
                    break
        return gSelec
    
    def cruce(self,A):
        result = []
        for i in range (0,len(A),2):
            matriz = np.random.randint(len(A[i].weights))
            aux = random.randint(1,A[i].weights[matriz].shape[1]-1)
            padre = A[i].weights[matriz]
            madre = A[i+1].weights[matriz]
            hijo1 = copy.deepcopy(A[i])
            hijo2 = copy.deepcopy(A[i+1])
            hijo_matriz1 = copy.deepcopy(padre)
            hijo_matriz1[:, aux:] = madre[:, aux:]
            hijo_matriz2 = copy.deepcopy(madre)
            hijo_matriz2[:, aux:] = padre[:, aux:]
            hijo1.weights[matriz] = hijo_matriz1
            hijo2.weights[matriz] = hijo_matriz2
            #print(padre, madre)
            #print(hijo_matriz1, hijo_matriz2)
            
            result.append(hijo1)
            result.append(hijo2)


        return result

    def unirBloques(self, A,B,g):
        for i in range (0,len(A)):
            g.append(A[i])
        for i in range(0,len(B)):
            g.append(B[i])
        return g
    def crossover(self):
        A = [] #cruzan
        aCont = 0
        B = [] #no cruzan
        bCont = 0
        for i in self.nets:
            decision = random.uniform(0, 1)
            if decision <= self.Pc:
                A.append(i)
            else:
                B.append(i)
        if len(A)%2 != 0:
            B.append(A.pop(len(A)-1))
        A_new = self.cruce(A)
        gCros = []
        gCros = self.unirBloques(A_new,B,gCros)
        return gCros
    def mutacion(self):
        new = copy.deepcopy(self.nets)
    
        for individuo in range(len(self.nets)):
            for matriz in range(len(self.nets[individuo].weights)):
                for fila in range(len(self.nets[individuo].weights[matriz])):
                    for elemento in range(len(self.nets[individuo].weights[matriz][fila])):
                        decision = random.uniform(0, 1)
                        if decision <= self.Pm:
                            new[individuo].weights[matriz][fila][elemento] += random.uniform(-0.5, 0.5)

        return new
    def evolve(self):
        self.nets = self.seleccion()
        self.nets = self.crossover()
        self.nets= self.mutacion()
    

In [149]:
def preprocess_data(data):
    proc_data = data.drop(['PassengerId'], 1)

    #rellenamos los valores vacios
    proc_data['Age'].fillna(proc_data['Age'].mean(), inplace=True)
    proc_data['Age'] = proc_data['Age'].astype(int)
    proc_data['Fare'] = proc_data['Fare'].interpolate()
    proc_data['Cabin'].fillna('U', inplace=True)

    #normalizamos las columnas
    proc_data['Title'] = pd.Series((name.split('.')[0].split(',')[1].strip() for name in data['Name']), index=data.index)
    proc_data['Title'] = proc_data['Title'].replace(['Lady', 'the Countess','Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    proc_data['Title'] = proc_data['Title'].map({"Master":0, "Miss":1, "Ms" : 1 , "Mme":1, "Mlle":1, "Mrs":1, "Mr":2, "Rare":3})

    proc_data['Sex'] = proc_data['Sex'].map({'male': 0, 'female': 1})
    proc_data['Embarked'] = proc_data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

        #Creation of a deck column corresponding to the letter contained in the cabin value
    proc_data['Cabin'] = proc_data['Cabin'].str[:1]
    proc_data['Cabin'] = proc_data['Cabin'].map({cabin: p for p, cabin in enumerate(set(cab for cab in proc_data['Cabin']))})

    proc_data = proc_data.drop(['Name', 'Ticket','Cabin','Title'], 1) 

    proc_data.dropna(axis =0, inplace=True)

    return proc_data

In [155]:


path = '/Users/helena.ridocci/Documents/Titanic'
X_data = pd.read_csv(path+'/train.csv')
X_val = pd.read_csv(path+'/test.csv')
x_data =preprocess_data(X_data)
x_val = preprocess_data(X_val)
y_data = x_data.iloc[:,0]
x_data = x_data.iloc[:,1:]
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size =0.2)
x_train = x_train.values
y_train = y_train.values
x_test = x_test.values
y_test = y_test.values

N= 10
net_size = [x_train.shape[1],6,5,2]
print(x_train.shape)
Pm = 0.001
Pc = 0.6
n_casillas = 100
ga = GA(N, net_size, Pc, Pm, x_train, y_train, n_casillas)
start_time = time.time()
for i in range(100):

    if i % 10 == 0:
        print("Current iteration : {}".format(i+1))
        print("Time taken by far : %.1f seconds" % (time.time() - start_time))
        print("Mean network accuracy: %.2f%%" % np.mean(ga.get_aptitudes()))
        print("Test values: %.2f%%\n" % ga.fitness(ga.nets[0], x_test, y_test))

        # evolve the population
    ga.evolve()



(711, 7)
Current iteration : 1
Time taken by far : 0.0 seconds


C:\Users\helena.ridocci\AppData\Local\Continuum\anaconda3\envs\env\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


Mean network accuracy: 0.72%
Test values: 0.18%

Current iteration : 11
Time taken by far : 2.6 seconds
Mean network accuracy: 1.00%
Test values: 1.00%

Current iteration : 21
Time taken by far : 5.2 seconds
Mean network accuracy: 1.00%
Test values: 1.00%

Current iteration : 31
Time taken by far : 7.8 seconds
Mean network accuracy: 1.00%
Test values: 1.00%

Current iteration : 41
Time taken by far : 10.5 seconds
Mean network accuracy: 1.00%
Test values: 1.00%

Current iteration : 51
Time taken by far : 13.2 seconds
Mean network accuracy: 1.00%
Test values: 1.00%

Current iteration : 61
Time taken by far : 16.2 seconds
Mean network accuracy: 1.00%
Test values: 1.00%

Current iteration : 71
Time taken by far : 18.9 seconds
Mean network accuracy: 1.00%
Test values: 1.00%

Current iteration : 81
Time taken by far : 21.7 seconds
Mean network accuracy: 1.00%
Test values: 1.00%

Current iteration : 91
Time taken by far : 24.3 seconds
Mean network accuracy: 1.00%
Test values: 1.00%

